# When will coronavirus peak?

## We are in a serious war against an invisible enemy.




https://www.talktomira.com/post/when-will-coronavirus-peak-and-end-social-distancing

Today our landscaper asked my wife, “When do you think the virus thing will end?” Did the landscaper think that the wife of a data scientist should know how to forecast this? Or, it is just a safe and hot chat topic for anybody nowadays.

Technically, the peak can be defined as the maximum hospital resource utilization and deaths associated with COVID-19. Researchers have been developing models to forecast when the virus will peak (https://www.talktomira.com/post/when-will-coronavirus-peak-and-end-social-distancing). To be honest, even as a data scientist, it is hard for me to validate how accurate their model forecasts are. For the general audience, we don’t have the knowledge and time to gather all this information. But one thing we know for sure is that the peak highly correlates to the most confirmed cases of Coronavirus within your state or city.

I would like to introduce a common technique that has been widely used in stock market analysis to check if the virus has reached its peak in your state or not.

## Moving Average Crossover Strategy

If you have ever traded any stocks, options, or bitcoins, you shall be familiar with the moving average crossover strategy. If not, you can check this resource: https://www.investopedia.com/articles/active-trading/052014/how-use-moving-average-buy-stocks.asp.
It is a straightforward, popular, and compelling technique in stock market analysis to examine the stock price moving average (MA). The moving averages smooth out noises of the price changes, and crossovers of price and MAs are one of the main MA strategies. The price crossing above or below a moving average (MA) gives a signal of a potential change in trend.

New Confirmed Cases MA Crossover check

The MA crossover strategy inspires me to apply a daily new confirmed cases MA crossover approach to check if the virus has reached its peak. I am going to share how I develop the solution in Python Jupyter Notebook.

Load the packages for analysis.

In [1]:
import os
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [17]:
path = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/'
file = 'us-states.csv'
df_us_states = pd.read_csv(os.path.join(path, file), parse_dates= ['date'])
df_us_states['date'] = df_us_states['date'].dt.date
df_us_states_sum = df_us_states.groupby(['state', 'date'])[['cases', 'deaths']].sum()
df_us_states_sum['new_confirmed'] = df_us_states_sum.cases.diff()
df_us_states_sum = df_us_states_sum.dropna()
df_us_states_sum['new_confirmed'] = np.where((df_us_states_sum.new_confirmed <  0), 0 ,df_us_states_sum.new_confirmed)
df_us_states_sum = df_us_states_sum.loc[df_us_states_sum.index.get_level_values(1) >= dt.date(2020, 3,10)]
df_comb_sum = df_us_states_sum.reset_index().set_index('date')

In [18]:
# create a list to store all the state names
states = df_comb_sum.state.unique()
import plotly.graph_objects as go
import ipywidgets
from ipywidgets import widgets
@ipywidgets.interact
def plot(state= widgets.Dropdown(options=states,value="New York", description='State:',disabled=False,), \
         ma_window=widgets.IntSlider(min=3, max=15, step=1, value=7, description="SMA window:")):
    df = df_comb_sum.loc[df_comb_sum['state'] == state]
    df['new_cases_sma'] = df['new_confirmed'].rolling(window=ma_window).mean()
    fig = go.Figure()
    fig.add_trace(go.Scatter(
            x=df.index,
            y=df['new_confirmed'],
            mode="lines",
            name="Daily New Confirmed Cases"
        ))
    fig.add_trace(go.Scatter(
            x=df.index,
            y=df['new_cases_sma'],
            mode="lines", 
            name= "Daily New Cases SMA{} ".format(ma_window)
        ))

    fig.show()

interactive(children=(Dropdown(description='State:', index=34, options=('Alabama', 'Alaska', 'American Samoa',…